In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df=pd.read_csv('/kaggle/input/russia-real-estate-20182021/all_v2.csv')

In [ ]:
df

In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
import datetime

In [ ]:
df['date']=pd.to_datetime(df['date'])

In [ ]:
df['time']=pd.to_datetime(df['time'],format='%H:%M:%S').dt.time

In [ ]:
df.describe()

In [ ]:
df.isnull().sum()

In [ ]:
df.describe()

In [ ]:
df1 = df.sort_values('price', ascending=False)

In [ ]:
df1

Drop the price below 0 and above 20.000000e+06, and rooms below 0
Sometimes, it is very difficult for us to estimate very high price in the mid city because it is evaluateded by any other reasons. So I doropped the price above 20.000000e+06.

In [ ]:
df[df['price']>=20.000000e+06]

In [ ]:
df[df['price'] <= 0]

In [ ]:
df[df['rooms'] <= 0]

In [ ]:
price=df[df['price'] <= 0].index
df.drop(price,inplace=True)

In [ ]:
rooms=df[df['rooms']<= 0].index
df.drop(rooms,inplace=True)

In [ ]:
price_high=df[df['price']>=20.000000e+06].index
df.drop(price_high,inplace=True)

In [ ]:
df.corr()

In [ ]:
sns.heatmap(df.corr())

LR, Lasso and Ridge Prediction model 

In [ ]:
from sklearn.linear_model import Ridge,Lasso,LinearRegression

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score
from sklearn.model_selection import KFold

In [ ]:
x=df.drop(['date', 'time','price'], axis=1)
y=df['price']

In [ ]:
x_train,x_test,y_train,y_test=train_test_split(x, y,train_size=0.8,random_state=123)

In [ ]:
model1=LinearRegression()
model2=Lasso(alpha=1.0)
model3=Ridge(alpha=0.1)

In [ ]:
scores=[]
kf=KFold(n_splits=4,shuffle=True,random_state=71)
for tr_idx,va_idx in kf.split(x_train):
    tr_x,va_x=x_train.iloc[tr_idx],x_train.iloc[va_idx]
    tr_y,va_y=y_train.iloc[tr_idx],y_train.iloc[va_idx]
    
    model1.fit(tr_x,tr_y)
    va_pred1=model1.predict(va_x)
    score_rmse1=np.sqrt(mean_squared_error(va_y,va_pred1)).mean()
    score_mae1=mean_absolute_error(va_y,va_pred1).mean()
    score_r21=r2_score(va_y,va_pred1).mean()

print('rmse1:',score_rmse1)
print('mae1:',score_mae1)
print('R21:',score_r21)

In [ ]:
scores=[]
kf=KFold(n_splits=4,shuffle=True,random_state=71)
for tr_idx,va_idx in kf.split(x_train):
    tr_x,va_x=x_train.iloc[tr_idx],x_train.iloc[va_idx]
    tr_y,va_y=y_train.iloc[tr_idx],y_train.iloc[va_idx]
    
    model2.fit(tr_x,tr_y)
    va_pred2=model2.predict(va_x)
    score_rmse2=np.sqrt(mean_squared_error(va_y,va_pred2)).mean()
    score_mae2=mean_absolute_error(va_y,va_pred2).mean()
    score_r22=r2_score(va_y,va_pred2).mean()

print('rmse2:',score_rmse2)
print('mae2:',score_mae2)
print('R22:',score_r22)

In [ ]:
scores=[]
kf=KFold(n_splits=4,shuffle=True,random_state=71)
for tr_idx,va_idx in kf.split(x_train):
    tr_x,va_x=x_train.iloc[tr_idx],x_train.iloc[va_idx]
    tr_y,va_y=y_train.iloc[tr_idx],y_train.iloc[va_idx]
    
    model3.fit(tr_x,tr_y)
    va_pred3=model3.predict(va_x)
    score_rmse3=np.sqrt(mean_squared_error(va_y,va_pred3)).mean()
    score_mae3=mean_absolute_error(va_y,va_pred3).mean()
    score_r23=r2_score(va_y,va_pred3).mean()

print('rmse3:',score_rmse3)
print('mae3:',score_mae3)
print('R23:',score_r23)

LightGBM model

In [ ]:
import lightgbm as lgb

In [ ]:
lgb_train = lgb.Dataset(x_train, y_train)
lgb_eval = lgb.Dataset(x_test, y_test)

In [ ]:
params={'metric': 'rmse'}

In [ ]:
gbm = lgb.train(params,
                lgb_train,
                valid_sets=lgb_eval,
                num_boost_round=5000,
                early_stopping_rounds=100,
                verbose_eval=100)

In [ ]:
predicted = gbm.predict(x_test)

In [ ]:
def pred_map(pred_df):
    rmse = np.sqrt(mean_squared_error(pred_df['data'], pred_df['pred']))
    R2 = r2_score(pred_df['data'], pred_df['pred']) 
    plt.figure(figsize=(8,8))
    ax = plt.subplot(111)
    ax.scatter('data', 'pred', data=pred_df)
    ax.set_xlabel('Data Value', fontsize=15)
    ax.set_ylabel('Pred Value', fontsize=15)
    ax.set_xlim(pred_df.min().min()-0.1 , pred_df.max().max()+0.1)
    ax.set_ylim(pred_df.min().min()-0.1 , pred_df.max().max()+0.1)
    x = np.linspace(pred_df.min().min()-0.1, pred_df.max().max()+0.1, 2)
    y = x
    ax.plot(x,y,'r-')
    plt.text(0.1, 0.9, 'rmse = {}'.format(str(round(rmse, 5))), transform=ax.transAxes, fontsize=15)
    plt.text(0.1, 0.8, 'R2 = {}'.format(str(round(R2, 5))), transform=ax.transAxes, fontsize=15)

In [ ]:
df_prediction = pd.concat([y_test.reset_index(drop=True), pd.Series(predicted)], axis=1)
df_prediction.columns = ['data', 'pred']

In [ ]:
pred_map(df_prediction)

In [ ]:
lgb.plot_importance(gbm, height=0.5, figsize=(8,16))

By LGBM without tuning, I can get more accurate model than LR,Lasso and Ridge.
RMSE=906567
R2=0.91014